In [175]:
import json
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from scipy.stats import pearsonr

cached_stopwords = stopwords.words("english")

In [176]:
input_file = 'data/alextanhongpin.json'

count_vec = CountVectorizer()
dict_vec = DictVectorizer()

def xstr(s):
    if s is None:
        return ''
    return s

items = {}
with open(input_file) as f:
    data = json.load(f)
    for item in data['data']:
        if items.get(item['login']) is None:
            items[item['login']] = (item['login'], [], [])
        keywords = xstr(item['name']) + ' ' + xstr(item['description']) + ' ' + xstr(item['language'])
        X_text = re.sub('\W+', ' ', keywords).lower().split(' ')
        X_text = [text for text in X_text if text not in cached_stopwords]
        if languages.get(item['language']) is None:
            languages[item['language']] = 0
        languages[item['language']] += 1

        # Second tuple
        [items[item['login']][1].append(txt) for txt in X_text]
        
        # Third tuple
#         items[item['login']][2].append(X_dict)



# Handle the count vector first
out = count_vec.fit_transform([' '.join(items[item][1]) 
                         for key, item in enumerate(items)]).toarray()
# print(count_vec.get_feature_names())
print(out)

print(pearsonr(out[0], out[1])[1])

[[ 1  2  0  4  1  1  1  2  1  1  1  0  2  1  3  2  0  0  1  2  2  2  0  2
   2  2  0  2  2  3  1  1  1  1  1  1  1  1  2  2  1  1  1  1  2  2  0  1
   1  1  0  3  2  1  2  2  0  2  4  2  1  0  2  0  2  0  0  1  0  2  1  3
   1  2  0  0  1  0  2  2  1  1  0  1  2  3  0  1  1  2  1 11  1  6  1  1
   2  1  2  0  0  0  1  0  0  1  1  1  1  2  1  1  2  0  0  0  1  2  0  1
   1  1  2  1  1  1  1  3  1  0  0  1  1  2  1  3  3  0  0  2  1  1  1  2
   2  1  1  5  0  2  0  2  0  3  2  1  1  0  2  1  1  1  0  1  1  1  2  0
   0  1  1]
 [ 0  0  1  0  0  0  0  0  1  0  0  2  0  0  0  0  2  1  0  0  0  0  1  0
   0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0
   0  0  1  0  0  0  0  0  2  0  0  0  1  1  0  1  0  1  1  0  5  3  0  0
   0  0  1  1  0  2  0  3  3  0  1  0  2  0  1  0  0  0  1  1  0  0  0  0
   0  0  0  1  1  2  0  1  1  0  0  0  0  0  0  0  0  2  1  1  0  0  1  0
   0  0  0  0  0  0  0  0  0  1  1  0  0  0  0  0  0  1  3  0  0  0  0  0
   0  0  0  0  1  0  1  0 